NEURAL NETWORK **PREDICTION**

Importing

In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import seed
seed(1)
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

import tensorflow
tensorflow.random.set_seed(1)
from tensorflow.python.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

Data Preprocessing

In [8]:
train_df = pd.read_csv("/content/H1.csv")
a=train_df.head()
b=train_df
b
b.sort_values(['ArrivalDateYear','ArrivalDateWeekNumber'], ascending=True)

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,...,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate
0,0,342,2015,July,27,1,0,0,2,0,...,No Deposit,NULL,NULL,0,Transient,0.00,0,0,Check-Out,2015-07-01
1,0,737,2015,July,27,1,0,0,2,0,...,No Deposit,NULL,NULL,0,Transient,0.00,0,0,Check-Out,2015-07-01
2,0,7,2015,July,27,1,0,1,1,0,...,No Deposit,NULL,NULL,0,Transient,75.00,0,0,Check-Out,2015-07-02
3,0,13,2015,July,27,1,0,1,1,0,...,No Deposit,304,NULL,0,Transient,75.00,0,0,Check-Out,2015-07-02
4,0,14,2015,July,27,1,0,2,2,0,...,No Deposit,240,NULL,0,Transient,98.00,0,1,Check-Out,2015-07-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40055,0,212,2017,August,35,31,2,8,2,1,...,No Deposit,143,NULL,0,Transient,89.75,0,0,Check-Out,2017-09-10
40056,0,169,2017,August,35,30,2,9,2,0,...,No Deposit,250,NULL,0,Transient-Party,202.27,0,1,Check-Out,2017-09-10
40057,0,204,2017,August,35,29,4,10,2,0,...,No Deposit,250,NULL,0,Transient,153.57,0,3,Check-Out,2017-09-12
40058,0,211,2017,August,35,31,4,10,2,0,...,No Deposit,40,NULL,0,Contract,112.80,0,1,Check-Out,2017-09-14


Numerical variables

In [10]:
leadtime = train_df['LeadTime']
arrivaldateyear = train_df['ArrivalDateYear']
arrivaldateweekno = train_df['ArrivalDateWeekNumber']
arrivaldatedayofmonth = train_df['ArrivalDateDayOfMonth']
staysweekendnights = train_df['StaysInWeekendNights']
staysweeknights = train_df['StaysInWeekNights']
adults = train_df['Adults']
children = train_df['Children']
babies = train_df['Babies']
isrepeatedguest = train_df['IsRepeatedGuest'] 
previouscancellations = train_df['PreviousCancellations']
previousbookingsnotcanceled = train_df['PreviousBookingsNotCanceled']
bookingchanges = train_df['BookingChanges']
agent = train_df['Agent']
company = train_df['Company']
dayswaitinglist = train_df['DaysInWaitingList']
adr = train_df['ADR']
rcps = train_df['RequiredCarParkingSpaces']
totalsqr = train_df['TotalOfSpecialRequests']

In [12]:
y1 = np.array(adr)

Categorical Variables

In [14]:
IsCanceled = train_df['IsCanceled']
arrivaldatemonth = train_df.ArrivalDateMonth.astype("category").cat.codes
arrivaldatemonthcat=pd.Series(arrivaldatemonth)
mealcat=train_df.Meal.astype("category").cat.codes
mealcat=pd.Series(mealcat)
countrycat=train_df.Country.astype("category").cat.codes
countrycat=pd.Series(countrycat)
marketsegmentcat=train_df.MarketSegment.astype("category").cat.codes
marketsegmentcat=pd.Series(marketsegmentcat)
distributionchannelcat=train_df.DistributionChannel.astype("category").cat.codes
distributionchannelcat=pd.Series(distributionchannelcat)
reservedroomtypecat=train_df.ReservedRoomType.astype("category").cat.codes
reservedroomtypecat=pd.Series(reservedroomtypecat)
assignedroomtypecat=train_df.AssignedRoomType.astype("category").cat.codes
assignedroomtypecat=pd.Series(assignedroomtypecat)
deposittypecat=train_df.DepositType.astype("category").cat.codes
deposittypecat=pd.Series(deposittypecat)
customertypecat=train_df.CustomerType.astype("category").cat.codes
customertypecat=pd.Series(customertypecat)
reservationstatuscat=train_df.ReservationStatus.astype("category").cat.codes
reservationstatuscat=pd.Series(reservationstatuscat)

In [16]:
x1 = np.column_stack((IsCanceled,countrycat,marketsegmentcat,deposittypecat,customertypecat,rcps,arrivaldateweekno))
x1 = sm.add_constant(x1, prepend=True)

Training and Validation Data Partition

In [18]:
X_train, X_val, y_train, y_val = train_test_split(x1, y1)

In [20]:
#Variables
y_train=np.reshape(y_train, (-1,1))
y_val=np.reshape(y_val, (-1,1))

scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

print(scaler_x.fit(X_train))
xtrain_scale=scaler_x.transform(X_train)
print(scaler_x.fit(X_val))
xval_scale=scaler_x.transform(X_val)

print(scaler_y.fit(y_train))
ytrain_scale=scaler_y.transform(y_train)
print(scaler_y.fit(y_val))
yval_scale=scaler_y.transform(y_val)

MinMaxScaler()
MinMaxScaler()
MinMaxScaler()
MinMaxScaler()


In [22]:
ytrain_scale

array([[0.35845095],
       [0.21361639],
       [0.14926708],
       ...,
       [0.12516039],
       [0.09026401],
       [0.15820988]])

In [24]:
yval_scale

array([[0.19402985],
       [0.08034826],
       [0.12935323],
       ...,
       [0.44776119],
       [0.11940299],
       [0.75870647]])

In [26]:
xtrain_scale

array([[0.        , 0.        , 0.768     , ..., 0.66666667, 0.        ,
        0.5       ],
       [0.        , 0.        , 0.        , ..., 0.66666667, 0.        ,
        0.73076923],
       [0.        , 0.        , 0.368     , ..., 0.        , 0.        ,
        0.44230769],
       ...,
       [0.        , 0.        , 0.328     , ..., 0.66666667, 0.        ,
        0.11538462],
       [0.        , 1.        , 0.368     , ..., 0.66666667, 0.        ,
        0.71153846],
       [0.        , 0.        , 0.768     , ..., 0.66666667, 0.125     ,
        0.11538462]])

In [28]:
xval_scale

array([[0.        , 1.        , 0.36585366, ..., 0.66666667, 0.        ,
        0.23076923],
       [0.        , 1.        , 0.7804878 , ..., 0.66666667, 0.        ,
        0.01923077],
       [0.        , 0.        , 0.17886179, ..., 1.        , 0.        ,
        0.07692308],
       ...,
       [0.        , 1.        , 0.37398374, ..., 0.66666667, 0.        ,
        0.48076923],
       [0.        , 1.        , 0.7804878 , ..., 0.66666667, 0.        ,
        0.17307692],
       [0.        , 1.        , 0.60162602, ..., 0.66666667, 0.        ,
        0.55769231]])

Neural Network Configuration

In [30]:
model = Sequential()
model.add(Dense(8, input_dim=8, kernel_initializer='normal', activation='relu'))
model.add(Dense(2670, activation='relu'))
model.add(Dense(1, activation='linear'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_4 (Dense)              (None, 2670)              24030     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 2671      
Total params: 26,773
Trainable params: 26,773
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])
history=model.fit(xtrain_scale, ytrain_scale, epochs=30, batch_size=150, verbose=1, validation_split=0.2)
predictions = model.predict(xval_scale)

Epoch 1/30
161/161 [==============================] - 4s 16ms/step - loss: 0.0046 - mse: 0.0046 - mae: 0.0479 - val_loss: 0.0047 - val_mse: 0.0047 - val_mae: 0.0470
Epoch 2/30
161/161 [==============================] - 2s 14ms/step - loss: 0.0047 - mse: 0.0047 - mae: 0.0480 - val_loss: 0.0047 - val_mse: 0.0047 - val_mae: 0.0468
Epoch 3/30
161/161 [==============================] - 2s 13ms/step - loss: 0.0046 - mse: 0.0046 - mae: 0.0477 - val_loss: 0.0054 - val_mse: 0.0054 - val_mae: 0.0490
Epoch 4/30
161/161 [==============================] - 1s 7ms/step - loss: 0.0046 - mse: 0.0046 - mae: 0.0480 - val_loss: 0.0048 - val_mse: 0.0048 - val_mae: 0.0473
Epoch 5/30
161/161 [==============================] - 1s 8ms/step - loss: 0.0046 - mse: 0.0046 - mae: 0.0479 - val_loss: 0.0048 - val_mse: 0.0048 - val_mae: 0.0473
Epoch 6/30
161/161 [==============================] - 1s 7ms/step - loss: 0.0046 - mse: 0.0046 - mae: 0.0477 - val_loss: 0.0047 - val_mse: 0.0047 - val_mae: 0.0473
Epoch 7/30
16

Accuracy on validation set

In [38]:
predictions = scaler_y.inverse_transform(predictions)
predictions


array([[27165.096],
       [19685.607],
       [19969.053],
       ...,
       [51678.875],
       [21766.52 ],
       [58689.926]], dtype=float32)

In [40]:
mean_absolute_error(y_val, predictions)

33088.44316837087

In [42]:
mean_squared_error(y_val, predictions)
math.sqrt(mean_squared_error(y_val, predictions))

36769.095431450005

In [44]:
np.mean(y_val)

94.62536295556666

In [45]:
np.mean(predictions)

33179.703

Neural Network

In [46]:
class nn:
    def __init__(self,ip,op):
        self.ip=ip
        self.op=op
        print(ip.shape,op.shape)
        self.l1_size=100
        self.w1=(np.random.rand(self.l1_size,self.ip.shape[1]+1).T)*0.3-0.15
        self.w2=(np.random.rand(self.op.shape[1],self.l1_size+1).T)*0.3-0.15
        print(self.w1.shape,self.w2.shape)
        for i in range(50):
            print(i)
            self.train()
    
    def sig(self,x):
        return 1/(1+np.exp(-x))
    
    def train(self):
        #===front propogation===#
        ip=self.sig(self.ip)
        ip=np.append(np.ones((len(ip),1)),ip,axis=1)
        l1=self.sig(ip@self.w1)
        l1=np.append(np.ones((len(l1),1)),l1,axis=1)
        pred=(l1@self.w2)
        #===back propogation===#
        #error at each layer
        d3 = (pred - self.op)
        d2 = (d3 @ self.w2.T * l1 * (1-l1))
        d2 = d2[:,1:]
        #altering weights
        self.w2-= (l1.T @ d3)/len(ip)
        self.w1-= (ip.T @ d2)/len(ip)
        
    def predict(self,ip):
        ip=np.append(np.ones((len(ip),1)),ip,axis=1)
        l1=self.sig(ip@self.w1)
        l1=np.append(np.ones((len(l1),1)),l1,axis=1)
        pred=self.sig(l1@self.w2)
        return pred

In [47]:
newn=nn(xtrain_scale,ytrain_scale)

(30045, 8) (30045, 1)
(9, 100) (101, 1)
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


Comparison

In [49]:
prediction=newn.predict(xval_scale)

In [56]:
mean_absolute_error(yval_scale, predictions)

33182.82099342681

In [57]:
mean_absolute_error(yval_scale, prediction)

0.3002139147647217

In [59]:
prediction

array([[0.54522151],
       [0.54676613],
       [0.52167035],
       ...,
       [0.54700363],
       [0.54752377],
       [0.54821489]])

In [58]:
yval_scale

array([[0.19402985],
       [0.08034826],
       [0.12935323],
       ...,
       [0.44776119],
       [0.11940299],
       [0.75870647]])

Prediction rate gradually increases with increase in the number of iterations


-Lohesh M(21110290)

-Lohesh M(21110290)